# Exercise Synopsys

Before opening their APIs to the public, CitiBank's Chief Digital Officer is keen in exploring new revenue streams to monetize the larges amount of data they've been historically collecting. He knows other financial institutions are doing business with their data, but he still needs to define specific use cases in which his credit card transaction data can add real value. As Senior Data Scientists, the CDO assigns you with the key task of exploring in depth the data stored in your data centers and make a solid assessment of the real value provided by it. You will present this business case in Citibank's annual convention. 

# Imports

In [2]:
from plotnine import *
from plotnine.data import mpg
import pandas as pd
import numpy as np
%matplotlib inline

# Importing dataframe
    

In [3]:
df = pd.read_csv ("madrid_transactions.csv")

The dataset provided contains more than 10,000 credit card transactions performed by tourists in the city of Madrid during a month. CitiBank's DBA has already cleaned the file and uploaded it to the Campus Online, so you can solely focus on analyzing the different consumption patterns from tourists. The goal is to discover different insights from the data through data visualization (with GGPlot2) and then transform them into potential business outcomes. 

# Examples of Business Cases

- Is there a particular time in which tourists are buying?
- Which nationality has the highest average ticket and why?
- Is there any relationship between day of the week and consumption?
- Do high end fashion retailers need to focus more on attracting Australian visitors than on Chinese? Why? 

In [4]:
df.head()

,Unnamed: 0,amount,category,customer_country,daytime,hour,tx_date_proc,weekday
0,1,33.46,Fashion & Shoes,GB,Night,21,2012-03-01 21:51:54+00,thursday
1,2,59.97,Fashion & Shoes,GB,Afternoon,16,2012-03-02 16:12:52+00,friday
2,3,49.99,Fashion & Shoes,CN,Evening,17,2012-03-02 17:34:01+00,friday
3,4,5.95,Fashion & Shoes,GB,Evening,17,2012-03-01 17:43:29+00,thursday
4,5,16.65,Bars & restaurants,DK,Night,22,2012-03-01 22:47:52+00,thursday


In [5]:
df.tail()

,Unnamed: 0,amount,category,customer_country,daytime,hour,tx_date_proc,weekday
10788,10789,79.33,Bars & restaurants,IN,Night,21,2012-03-02 21:22:51+00,friday
10789,10790,77.70,Bars & restaurants,US,Night,22,2012-03-02 22:42:09+00,friday
10790,10791,48.25,Bars & restaurants,US,Midnight,23,2012-03-02 23:01:19+00,friday
10791,10792,368.90,Bars & restaurants,MY,Night,22,2012-03-02 22:35:46+00,friday
10792,10793,452.55,Bars & restaurants,CN,Afternoon,15,2012-03-01 15:27:26+00,thursday
